# oi

In [5]:
import os, sys, logging
sys.path.append(os.getcwd().split('paper_std')[0])
if '../' not in sys.path:
    sys.path.append('../')

from util.load_s3_data import LoadS3Data
from util.time_method import *
from data_generator import *
from typing import TypedDict
from feature import FeatureGenerator, FeatureConfig
from util.hedge_log import initlog


class OIGenerator(FeatureGenerator):
    '''
    利用ticker数据，生成订单不平衡（Order Imbalance）,不需要输入额外参数
    '''

    def __init__(self, config: FeatureConfig) -> None:
        super().__init__(config)

        self.fe_name = 'oi' 
        self.previous_bid_volume = None  
        self.previous_ask_volume = None
        self.previous_bid_price = None  
        self.previous_ask_price = None
    def process(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):
        
        if ticker == None:  #如果没有传来ticker数据，则不更新
            return 
        self.update(ticker)
        feature_ret = self.calculate(ticker)

        return feature_ret
    
    def update(self, ticker: QuoteTick):  
        self.current_bid_volume = ticker.bid_size * ticker.bid_price
        self.current_ask_volume = ticker.ask_size * ticker.ask_price
        self.current_bid_price = ticker.bid_price
        self.current_ask_price = ticker.ask_price
        self.ts_event = ticker.ts_event
        
    def calculate(self, ticker: QuoteTick):

        if self.previous_bid_volume is not None:
            if self.current_bid_price == self.previous_bid_price:
                delta_bid = self.current_bid_volume - self.previous_bid_volume
            elif self.current_bid_price > self.previous_bid_price:
                delta_bid = self.current_bid_volume
            elif self.current_bid_price < self.previous_bid_price:
                delta_bid = 0
            
            if self.current_ask_price == self.previous_ask_price:
                delta_ask = self.current_ask_volume - self.previous_ask_volume
            elif self.current_ask_price < self.previous_ask_price:
                delta_ask = self.current_ask_volume
            elif self.current_ask_price > self.previous_ask_price:
                delta_ask = 0
            
            oi = delta_bid - delta_ask
        else:
            oi = None

        self.previous_bid_volume = self.current_bid_volume
        self.previous_ask_volume = self.current_ask_volume
        self.previous_bid_price = self.current_bid_price
        self.previous_ask_price = self.current_ask_price
        
        return [(self.fe_name, self.ts_event, oi)]


if __name__ == '__main__':     
    initlog(None, 'oi_feature.log', logging.INFO)
    start = time.perf_counter() 
    #不需要额外参数
    oi_config = FeatureConfig()
    print()

    begin_time = datetime.datetime(2024, 3, 27, 21,0, tzinfo=TZ_8) # 要加一个tzinfo
    end_time = datetime.datetime(2024, 3, 27, 21, 59,tzinfo=TZ_8)
    exchange = 'binance'
    symbol = 'btc_usdt'

    ins = OIGenerator(oi_config)
    fe_list = []

    data_generator = get_data_generator(begin_time, end_time, exchange, symbol)
    
    
    for idx, row in enumerate(data_generator):
        if row[1] == 'ticker':
            fe_list.append(ins.process(ticker=row[0]))
        elif row[1] == 'depth':
            fe_list.append(ins.process(depth=row[0]))
        else:
            fe_list.append(ins.process(trade=row[0]))

#         if idx > 10:
#             break
    print(fe_list[:100])
    end = time.perf_counter()   
    duration = end - start 
    print("程序运行时间：", duration)

None/oi_feature.log

2024-04-02 01:39:05.049 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 01:39:05.049 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 01:39:05.051 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-02 01:39:05.051 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-02 01:39:05.055 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032721 812399
2024-04-02 01:39:05.055 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032721 812399
2024-04-02 01:39:05.067 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032722 759724
2024-04-02 01:39:05.067 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据:

# logoi

In [8]:
import os, sys, logging
sys.path.append(os.getcwd().split('paper_std')[0])
if '../' not in sys.path:
    sys.path.append('../')

from util.load_s3_data import LoadS3Data
from util.time_method import *
from data_generator import *
from typing import TypedDict
from feature import FeatureGenerator, FeatureConfig
from util.hedge_log import initlog
import numpy as np

class LOGOIGenerator(FeatureGenerator):
    '''
    利用ticker数据，生成对数版订单不平衡（LOG Order Imbalance）,不需要输入额外参数
    '''

    def __init__(self, config: FeatureConfig) -> None:
        super().__init__(config)

        self.fe_name = 'logoi' 
        self.previous_bid_log_volume = None  
        self.previous_ask_log_volume = None
        self.previous_bid_price = None  
        self.previous_ask_price = None
    def process(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):
        
        if ticker == None:  #如果没有传来ticker数据，则不更新
            return 
        self.update(ticker)
        feature_ret = self.calculate(ticker)

        return feature_ret
    
    def update(self, ticker: QuoteTick):
        self.current_bid_log_volume = np.log10(ticker.bid_size)
        self.current_ask_log_volume = np.log10(ticker.ask_size)
        self.current_bid_price = ticker.bid_price
        self.current_ask_price = ticker.ask_price
        self.ts_event = ticker.ts_event
        
    def calculate(self, ticker: QuoteTick):

        if self.previous_bid_log_volume is not None:
            if self.current_bid_price == self.previous_bid_price:
                delta_bid = self.current_bid_log_volume - self.previous_bid_log_volume
            elif self.current_bid_price > self.previous_bid_price:
                delta_bid = self.current_bid_log_volume
            elif self.current_bid_price < self.previous_bid_price:
                delta_bid = - self.previous_bid_log_volume
            
            if self.current_ask_price == self.previous_ask_price:
                delta_ask = self.current_ask_log_volume - self.previous_ask_log_volume
            elif self.current_ask_price < self.previous_ask_price:
                delta_ask = self.current_ask_log_volume
            elif self.current_ask_price > self.previous_ask_price:
                delta_ask = - self.previous_ask_log_volume
            
            logoi = delta_bid - delta_ask
        else:
            logoi = None

        self.previous_bid_log_volume = self.current_bid_log_volume
        self.previous_ask_log_volume = self.current_ask_log_volume
        self.previous_bid_price = self.current_bid_price
        self.previous_ask_price = self.current_ask_price
        
        return [(self.fe_name, self.ts_event, logoi)]


if __name__ == '__main__':     
    initlog(None, 'logoi_feature.log', logging.INFO)
    logoi_config = FeatureConfig()

    begin_time = datetime.datetime(2024, 3, 27, 21,tzinfo=TZ_8) # 要加一个tzinfo
    end_time = datetime.datetime(2024, 3, 27, 21, 1,tzinfo=TZ_8)
    exchange = 'binance'
    symbol = 'btc_usdt'

    ins = LOGOIGenerator(ofi_config)
    fe_list = []

    data_generator = get_data_generator(begin_time, end_time, exchange, symbol)
    for idx, row in enumerate(data_generator):
        #print(idx)
        if row[1] == 'ticker':
            fe_list.append(ins.process(ticker=row[0]))
        elif row[1] == 'depth':
            fe_list.append(ins.process(depth=row[0]))
        else:
            fe_list.append(ins.process(trade=row[0]))

        if idx > 10:
            break
    print(fe_list)
  

INFO:root:开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据


None/logoi_feature.log
2024-04-01 08:05:04.995 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:05:04.995 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:05:04.995 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:05:04.995 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据


INFO:root:获取 binance_btc_usdt cex ticker数据: 2024032720 213409


2024-04-01 08:05:04.998 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:05:04.998 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:05:04.998 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:05:04.998 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409


INFO:root:获取 binance_btc_usdt cex ticker数据: 2024032721 812399


...

2024-04-01 08:05:05.223 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:05:05.223 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:05:05.223 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:05:05.223 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据


INFO:root:开始恢复 binance_btc_usdt 的depth数据


2024-04-01 08:05:05.318 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:05:05.318 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:05:05.318 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:05:05.318 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据


INFO:root:数据组装完成 共耗时: 2.4248950481414795s


2.4240317344665527
2.424848794937134
2.424886465072632
2024-04-01 08:05:07.423 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.4248950481414795s
2024-04-01 08:05:07.423 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.4248950481414795s
2024-04-01 08:05:07.423 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.4248950481414795s
2024-04-01 08:05:07.423 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.4248950481414795s
[[('logoi', 1711544400000000000, None)], None, None, None, None, [('logoi', 1711544400004000000, -0.0001642649480418923)], [('logoi', 1711544400004000000, -0.00015898215308673969)], [('logoi', 1711544400004000000, 0.0019352322138996847)], None, None, None, [('logoi', 1711544400007000000, -0.00021251240323227893)]]


# oir

In [11]:
import os, sys, logging
sys.path.append(os.getcwd().split('paper_std')[0])
if '../' not in sys.path:
    sys.path.append('../')

from util.load_s3_data import LoadS3Data
from util.time_method import *
from data_generator import *
from typing import TypedDict
from feature import FeatureGenerator, FeatureConfig
from util.hedge_log import initlog


class OIRGenerator(FeatureGenerator):
    '''
    利用ticker数据，生成订单不平衡率（Order Imbalance Ratio）,不需要输入额外参数
    '''

    def __init__(self, config: FeatureConfig) -> None:
        super().__init__(config)

        self.fe_name = 'oir'  # The feature name for bid price difference
        
    def process(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):
        
        if ticker == None:  #如果没有传来ticker数据，则不更新
            return 
        
        self.update(ticker)
        feature_ret = self.calculate()

        return feature_ret
    
    def update(self, ticker: QuoteTick):  
        self.current_bid_volume = ticker.bid_size
        self.current_ask_volume = ticker.ask_size
        self.ts_event = ticker.ts_event
        
    def calculate(self):

        oir = (self.current_bid_volume - self.current_ask_volume)/(self.current_bid_volume + self.current_ask_volume)
        
        return [(self.fe_name, self.ts_event, oir)]


if __name__ == '__main__':     
    initlog(None, 'oir_feature.log', logging.INFO)
    oir_config = FeatureConfig()
    print()

    begin_time = datetime.datetime(2024, 3, 27, 21,0,tzinfo=TZ_8) # 要加一个tzinfo
    end_time = datetime.datetime(2024, 3, 27, 21, 59,tzinfo=TZ_8)
    exchange = 'binance'
    symbol = 'btc_usdt'

    ins = OIRGenerator(oir_config)
    fe_list = []
    start = time.perf_counter() 
    data_generator = get_data_generator(begin_time, end_time, exchange, symbol)
    start = time.perf_counter() 
    for idx, row in enumerate(data_generator):
        #print(idx)
        if row[1] == 'ticker':
            fe_list.append(ins.process(ticker=row[0]))
        elif row[1] == 'depth':
            fe_list.append(ins.process(depth=row[0]))
        else:
            fe_list.append(ins.process(trade=row[0]))

#         if idx > 40:
#             break
#     print(fe_list)
    end = time.perf_counter()   
    duration = end - start 
    print("程序运行时间：", duration)

None/oir_feature.log

2024-04-02 02:30:44.914 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:30:44.914 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:30:44.914 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:30:44.914 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:30:44.914 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:30:44.914 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:30:44.915 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex t

# breadth

In [16]:
import os, sys, logging
sys.path.append(os.getcwd().split('paper_std')[0])
if '../' not in sys.path:
    sys.path.append('../')

from util.load_s3_data import LoadS3Data
from util.time_method import *
from data_generator import *
from typing import TypedDict
from feature import FeatureGenerator, FeatureConfig
from util.hedge_log import initlog


class BreadthGenerator(FeatureGenerator):


    def __init__(self, config: FeatureConfig) -> None:
        super().__init__(config)

        self.fe_name = 'Breadth'  
        self.config = config
        self.left = config['left']
        self.right = config['right']
        self.ticker_count = np.zeros(self.right+1)# 记录right次trade中间有多少次 ticker，初始都是0
        self.trade_num = -1 # 记录已经存储的交易次数
        self.last_trade_time = 0 
        self.ts_event = 0
    def process(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):

        self.update(ticker, trade, depth)
        feature_ret = self.calculate()

        return feature_ret
    
    def update(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):
        if trade != None: 
            self.ts_event = trade.ts_event
            if self.ts_event != self.last_trade_time:  # 判断交易时间是否相同，如果是同一个时刻发生的交易则不更新trade_num
                self.trade_num += 1 
                self.last_trade_time = self.ts_event
            if self.trade_num > self.right: # 如果大于endtime，将第一个元素删除，并在最后补一个0
                self.ticker_count = np.append(self.ticker_count[1:], 0)
                self.trade_num -= 1
        elif ticker != None: # ticker数据只更新状态，不返回信息
            self.ts_event = ticker.ts_event
            if self.trade_num >= 0:
                self.ticker_count[self.trade_num] += 1 
            return 
        elif depth != None: # 不处理depth数据
            self.ts_event = depth.ts_event
            return 
        return 
    
    def calculate(self):   
        if self.trade_num >= self.right: # 如果到right，计算并返回breadth值
            breadth = np.sum(self.ticker_count[0:(self.right-self.left)])
        else:  # 如果没有到right，返回None
            breadth = None
        return [(self.fe_name, self.ts_event, breadth)]


if __name__ == '__main__':     
    initlog(None, 'breadth_feature.log', logging.INFO)
    breadth_config = FeatureConfig(left = 2, right = 4)

    begin_time = datetime.datetime(2024, 3, 27, 21,tzinfo=TZ_8) # 要加一个tzinfo
    end_time = datetime.datetime(2024, 3, 27, 21, 1,tzinfo=TZ_8)
    exchange = 'binance'
    symbol = 'btc_usdt'

    ins = BreadthGenerator(breadth_config)
    fe_list = []

    data_generator = get_data_generator(begin_time, right, exchange, symbol)
    for idx, row in enumerate(data_generator):
        #print(idx)
        if row[1] == 'ticker':
            fe_list.append(ins.process(ticker=row[0]))
        elif row[1] == 'depth':
            fe_list.append(ins.process(depth=row[0]))
        else:
            fe_list.append(ins.process(trade=row[0]))
        if idx > 100:
            break
    print(fe_list)
  

INFO:root:开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据


None/breadth_feature.log
2024-04-01 08:27:21.176 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:27:21.176 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:27:21.176 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:27:21.176 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:27:21.176 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:27:21.176 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:01:00+08:00的数据
2024-04-01 08:27:21.176 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc

INFO:root:获取 binance_btc_usdt cex ticker数据: 2024032720 213409


2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-01 08:27:21.182 load_s3_data.py get_cex_ticker 50 INFO -

INFO:root:获取 binance_btc_usdt cex ticker数据: 2024032721 812399


...

2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2024-04-01 08:27:21.413 load_s3_data.py get_cex_trade 365 INFO - 开始查看cex-trade: binance btc_usdt 2024032721的数据
2

INFO:root:开始恢复 binance_btc_usdt 的depth数据


2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开始恢复 binance_btc_usdt 的depth数据
2024-04-01 08:27:21.509 load_s3_data.py get_cex_depth_online 500 INFO - 开

INFO:root:数据组装完成 共耗时: 2.292872190475464s


2.2920472621917725
2.292828321456909
2.292862892150879
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:23.474 data_generator.py get_data_generator 118 INFO - 数据组装完成 共耗时: 2.292872190475464s
2024-04-01 08:27:2

In [4]:
import os, sys, logging
sys.path.append(os.getcwd().split('paper_std')[0])
if '../' not in sys.path:
    sys.path.append('../')

from util.load_s3_data import LoadS3Data
from util.time_method import *
from data_generator import *
from typing import TypedDict
from feature import FeatureGenerator, FeatureConfig
from util.hedge_log import initlog
import time

class BreadthGenerator(FeatureGenerator):

    def __init__(self, config: FeatureConfig) -> None:
        super().__init__(config)
        self.left = config['left']
        self.right = config['right']
        self.ticker_count = np.zeros(self.right+1)# 记录right次trade中间有多少次 ticker，初始都是0
        self.trade_volume = np.zeros(self.right+1)# 记录right个时刻trade交易量之和，初始为0
        self.trade_num = -1 # 记录已经存储的交易次数
        self.last_trade_time = 0 
        self.ts_event = 0
        self.fe_name1 = f'Breadth_{self.left}_{self.right}'  
        self.fe_name2 = f'Immediacy_{self.left}_{self.right}'
        self.fe_name3 = f'VolumeAvg_{self.left}_{self.right}'
        
    def process(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):

        self.update(ticker, trade, depth)
        feature_ret = self.calculate()

        return feature_ret
    
    def update(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):
        if trade != None: 
            self.ts_event = trade.ts_event
            if self.ts_event != self.last_trade_time:  # 判断交易时间是否相同，如果是同一个时刻发生的交易则不更新trade_num
                self.trade_num += 1 
                self.last_trade_time = self.ts_event
            if self.trade_num > self.right: # 如果大于endtime，将第一个元素删除，并在最后补一个0
                self.ticker_count = np.append(self.ticker_count[1:], 0)
                self.trade_volume = np.append(self.trade_volume[1:], 0)
                self.trade_num -= 1
            self.trade_volume[self.trade_num] += trade.size
        elif ticker != None: # ticker数据只更新状态，不返回信息
            self.ts_event = ticker.ts_event
            if self.trade_num >= 0:
                self.ticker_count[self.trade_num] += 1 
            return 
        elif depth != None: # 不处理depth数据
            self.ts_event = depth.ts_event
            return 
        return 
    
    def calculate(self):   
        if self.trade_num >= self.right: # 如果到right，计算并返回breadth值
            breadth = sum(self.ticker_count[0:(self.right-self.left)])
            immediacy = (self.right-self.left) / breadth
            volumeavg = sum(self.trade_volume[0:(self.right-self.left)]) / breadth
        else:  # 如果没有到right，返回None
            breadth, immediacy, volumeavg = None, None, None
        return [(self.fe_name1, self.ts_event, breadth),
                (self.fe_name2, self.ts_event, immediacy),
                (self.fe_name3, self.ts_event, volumeavg)]


if __name__ == '__main__':     
    initlog(None, 'breadth_feature.log', logging.INFO)
    breadth_config = FeatureConfig(left = 0, right = 4)

    begin_time = datetime.datetime(2024, 3, 27, 21,0,tzinfo=TZ_8) # 要加一个tzinfo
    right = datetime.datetime(2024, 3, 27, 21, 59,tzinfo=TZ_8)
    exchange = 'binance'
    symbol = 'btc_usdt'

    ins = BreadthGenerator(breadth_config)
    fe_list = []

    data_generator = get_data_generator(begin_time, right, exchange, symbol)

 
    start = time.perf_counter() 
          
    
    for idx, row in enumerate(data_generator):
        #print(idx)
        if row[1] == 'ticker':
            fe_list.append(ins.process(ticker=row[0]))
        elif row[1] == 'depth':
            fe_list.append(ins.process(depth=row[0]))
        else:
            fe_list.append(ins.process(trade=row[0]))
#         if idx > 100:
#             break
#     print(fe_list)
    end = time.perf_counter()   
    duration = end - start 
    print("程序运行时间：", duration)

None/breadth_feature.log
2024-04-02 00:55:50.599 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 00:55:50.599 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 00:55:50.599 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 00:55:50.599 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 00:55:50.601 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-02 00:55:50.601 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-02 00:55:50.601 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-02 00:55:50.601 load_s3_

In [10]:
import os, sys, logging
if '../' not in sys.path:
    sys.path.append('../')

from data_generator import *
from feature import FeatureGenerator, FeatureConfig
from util.hedge_log import initlog
import time
class BreadthGenerator(FeatureGenerator):
    '''
    注意 时间点相同的trade只算一次
    '''
    def __init__(self, config: FeatureConfig) -> None:
        super().__init__(config)
        self.left = config['left']
        self.right = config['right']
        self.ticker_count = np.zeros(self.right+1)# 记录right次trade中间有多少次 ticker，初始都是0
        self.trade_volume = np.zeros(self.right+1)# 记录right个时刻trade交易量之和，初始为0
        self.trade_num = -1 # 记录已经存储的交易次数
        self.last_trade_time = 0 
        self.ts_event = 0
        self.fe_name1 = f'Breadth_{self.left}_{self.right}'  
        self.fe_name2 = f'Immediacy_{self.left}_{self.right}'
        self.fe_name3 = f'VolumeAvg_{self.left}_{self.right}'
        
    def process(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):
        if depth != None:  #如果没有传来ticker数据或者trade数据，则不更新
            return 
        self.update(ticker, trade, depth)
        feature_ret = self.calculate()

        return feature_ret
    
    def update(self, ticker: QuoteTick=None, trade: TradeTick=None, depth: OrderBook=None):
        if trade != None: 
            self.ts_event = trade.ts_event
            if self.ts_event != self.last_trade_time:  # 判断交易时间是否相同，如果是同一个时刻发生的交易则不更新trade_num
                self.trade_num += 1 
                self.last_trade_time = self.ts_event
            if self.trade_num > self.right: # 如果大于endtime，将第一个元素删除，并在最后补一个0
                self.ticker_count = np.append(self.ticker_count[1:], 0)
                self.trade_volume = np.append(self.trade_volume[1:], 0)
                self.trade_num -= 1
            self.trade_volume[self.trade_num] += trade.size
        elif ticker != None: # ticker数据只更新状态，不返回信息
            self.ts_event = ticker.ts_event
            if self.trade_num >= 0:
                self.ticker_count[self.trade_num] += 1 
            return 
        elif depth != None: # 不处理depth数据
            self.ts_event = depth.ts_event
            return 
        return 
    
    def calculate(self):   
        if self.trade_num >= self.right: # 如果到right，计算并返回breadth值
            breadth = sum(self.ticker_count[0:(self.right-self.left)])
            immediacy = (self.right-self.left) / breadth
            volumeavg = sum(self.trade_volume[0:(self.right-self.left)]) / breadth
        else:  # 如果没有到right，返回None
            breadth, immediacy, volumeavg = None, None, None
        return [(self.fe_name1, self.ts_event/1e6, breadth),
                (self.fe_name2, self.ts_event/1e6, immediacy),
                (self.fe_name3, self.ts_event/1e6, volumeavg)]


if __name__ == '__main__':     
    initlog(None, 'breadth_feature.log', logging.INFO)
    breadth_config = FeatureConfig(left = 8, right = 16)

    begin_time = datetime.datetime(2024, 3, 27, 21,0,tzinfo=TZ_8) # 要加一个tzinfo
    right = datetime.datetime(2024, 3, 27, 21, 59,tzinfo=TZ_8)
    exchange = 'binance'
    symbol = 'btc_usdt'

    ins = BreadthGenerator(breadth_config)
    fe_list = []

    data_generator = get_data_generator(begin_time, right, exchange, symbol)
    start = time.perf_counter()     
    
    for idx, row in enumerate(data_generator):
        #print(idx)
        if row[1] == 'ticker':
            fe_list.append(ins.process(ticker=row[0]))
        elif row[1] == 'depth':
            fe_list.append(ins.process(depth=row[0]))
        else:
            fe_list.append(ins.process(trade=row[0]))

    end = time.perf_counter()   
    duration = end - start 
    print("程序运行时间：", duration)


None/breadth_feature.log
2024-04-02 02:29:13.390 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:29:13.390 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:29:13.390 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:29:13.390 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:29:13.390 data_generator.py get_data_generator 106 INFO - 开始生成binance:btc_usdt 2024-03-27 21:00:00+08:00~2024-03-27 21:59:00+08:00的数据
2024-04-02 02:29:13.392 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2024-04-02 02:29:13.392 load_s3_data.py get_cex_ticker 50 INFO - 获取 binance_btc_usdt cex ticker数据: 2024032720 213409
2